In [115]:
import pandas as pd

In [125]:
def format_name(name):
    """
    Formats the given name by removing the domain part if it's an email address.
    
    Parameters:
    name (str): The name to be formatted.
    
    Returns:
    str: The formatted name.
    """
    if "@" in name:
        return name.split('@')[0]  # Keeps the DC universe email intact
    return name

In [126]:
file_path = 'Avengers_vs_Invaders_Challenge.xlsx'
df_contacts = pd.read_excel(file_path, sheet_name='Contacts')

superheroes = pd.unique(df_contacts[['attack_role', 'defense_role', 'healing_role']].values.ravel('K'))
superheroes = [format_name(hero) for hero in superheroes if pd.notna(hero)]

# Create a dictionary to hold tables for each superhero
superhero_tables = {hero: [] for hero in superheroes}

print(superhero_tables)

{'iron.man': [], 'thor': [], 'wasp': [], 'ant.man': [], 'hulk': [], 'capatain.america': [], 'hawkeye': [], 'attack_role': [], 'quicksilver': [], 'scarlet.witch': [], 'hercules': [], 'black.panther': [], 'vision': [], 'black.widow': [], 'spider.man': [], 'deadpool': [], 'star.brand': [], 'shang-chi': [], 'sentry': [], 'wolverine': [], 'echo': [], 'ares': [], 'stature': [], 'amadeus_cho': [], 'valkyrie': [], 'nova': [], 'iron.fist': [], 'power.women': [], 'doctor.strange': [], 'daredevil': [], 'storm': [], 'quake': [], 'red.hulk': [], 'captain.britain': [], 'defense_role': [], 'batman@dc-world.com': [], 'healing_role': [], 'agent.venom': [], 'havok': [], 'sunspot': [], 'smasher': [], 'rogue': [], 'abyss': [], 'nightmask': []}


In [127]:
def read_and_split_excel(file_path):
    """
    Reads the contacts data from the specified Excel file.
    
    Parameters:
    file_path (str): The path to the Excel file.
    
    Returns:
    pd.DataFrame: The DataFrame containing the contacts data.
    """
    try:
        # Attempt to load the Excel file
        df = pd.read_excel(file_path, sheet_name='Contacts')
        
        # Check if the DataFrame is empty
        if df.empty:
            raise ValueError("The DataFrame is empty!")
        
        # Identify all blank rows
        blank_rows = df.index[df.isnull().all(axis=1)].tolist()
        
        # Include the start and end of the DataFrame to capture all sections
        section_boundaries = [0] + blank_rows + [len(df)]
        
        # Split into sections, ensuring not to start a section with a blank row
        sections = [df.iloc[section_boundaries[i]+1:section_boundaries[i+1]] for i in range(len(section_boundaries) - 1)]
        sections = [section.reset_index(drop=True) for section in sections if not section.empty]
        
        return sections
        
    except FileNotFoundError:
        print("File not found. Make sure the file path is correct.")
        return []
    except ValueError as ve:
        print(ve)
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Specify your Excel file path
file_path = 'Avengers_vs_Invaders_Challenge.xlsx'

# Read the Excel file and split into sections
dataframes = read_and_split_excel(file_path)

# Print each section (DataFrame)
sections = []
for i, df in enumerate(dataframes, 1):
    if i != 1:
        new_header = df.iloc[0]  # Take the first row for the header
        df = df[1:]  # Take the data less the header row
        df.columns = new_header  # Set the header row as the DataFrame header
        df.reset_index(drop=True, inplace=True)
        sections.append(df.fillna('').astype(str))
    else:
        new_row = {'DE-Headquarter': 'aliens', 'attack_role': 'iron.man', 'defense_role': ' ', 'healing_role': ' '}
        df = df.append(new_row, ignore_index=True)
        sections.append(df.fillna('').astype(str))
    #print(f"DataFrame #{i}:\n{df}\n")

df_dict ={}
for i, df in enumerate(sections, 1):
    first_column_header = df.columns[0]  # This is 'Key'
    df_dict[first_column_header] = df
    #print(f"DataFrame #{i}:\n{df}\n")
    #print(f"Data dict #{i}:\n{df_dict}\n")

In [128]:
# Define the invader species
invader_species = ['aliens', 'predators', 'd&d_beholder', 'd&d_devil', 'd&d_lich',
                   'd&d_mind_flayer', 'd&d_vampire', 'd&d_red_dragon', 'd&d_hill_giant',
                   'd&d_treant', 'd&d_werewolf', 'd&d_yuan-ti']

# Define the headquarters
headquarters = ['DE-Headquarter', 'US-Headquarter', 'UK-Headquarter', 'Ocean-Headquarter', 
                'BRA-Headquarter', 'CAN-Headquarter', 'COL-Headquarter', 'IND-Headquarter', 
                'MEX-Headquarter', 'NLD-Headquarter', 'NOR-Headquarter', 'PAK-Headquarter', 
                'TUR-Headquarter', 'UAE-Headquarter', 'ARG-Headquarter', 'Aircraft-Headquarter']

def get_role_abbreviation(row, hero):
    roles = []
    if row['attack_role'] == hero:
        roles.append('A')
    if row['defense_role'] == hero:
        roles.append('D')
    if row['healing_role'] == hero:
        roles.append('H')
    return ''.join(roles)

def insert_empty_row(df, position):
    empty_row = pd.Series([''] * len(df.columns), index=df.columns)
    top = df.iloc[:position]
    bottom = df.iloc[position:]
    df = pd.concat([top, pd.DataFrame([empty_row]), bottom]).reset_index(drop=True)
    return df


In [130]:
# Generate tables for each superhero
for hero in superhero_tables:
    #print(hero)
    table_data = { hero : headquarters}
    #print(table_data)
    for species in invader_species:
        #print(species)
        roles = []
        for hq in headquarters:
            #print(hq)
            df_contacts = df_dict[hq]
            #print(df_contacts[hq])
            if species in df_contacts[hq].values:
                #print(df_contacts)
                row = df_contacts[(df_contacts[hq] == species) & 
                                  (df_contacts['attack_role'] == hero) | 
                                  (df_contacts['defense_role'] == hero) | 
                                  (df_contacts['healing_role'] == hero)]
                #print(row)
                if not row.empty:
                    role_abbr = get_role_abbreviation(row.iloc[0], hero)
                    roles.append(role_abbr)
                    #print(roles)
                else:
                    roles.append('')
            else:
                roles.append('')
            
        table_data[species] = roles
    
    superhero_table = pd.DataFrame(table_data)
    superhero_table = insert_empty_row(superhero_table, 1)
    print(superhero_table)
    path = 'D:/Coding-challenges/' + 'Task02_'+ hero + '.xlsx'
    with pd.ExcelWriter(path) as writer:
        superhero_table.to_excel(writer, index=False)
    
print(f"Data successfully saved!")

                iron.man aliens predators d&d_beholder d&d_devil d&d_lich  \
0         DE-Headquarter      A                                             
1                                                                           
2         US-Headquarter                                                    
3         UK-Headquarter                                                    
4      Ocean-Headquarter                                                    
5        BRA-Headquarter                                                    
6        CAN-Headquarter                                                    
7        COL-Headquarter                                                    
8        IND-Headquarter                                                    
9        MEX-Headquarter                                                    
10       NLD-Headquarter                                                    
11       NOR-Headquarter                                                    